#  Machine Learning and Causal Inference - Problem Set 2

By Mathieu Breier, Joaquin Ossa

Date: 19 February 2024
------------------------

### Objective: Calculating the ATE of the RHC dataset, implementing the Double ML technique with cross-fitting

+ Treatment (D): swang1
+ Outcome (Y): death
+ Confounders (X): confounder.yml


#### 1. Train ML model Y~X and calculate residuals using cross-fitting
- Execute cross-validation (train-test split) over a subset of hyperparameters, and choose the one with lower error. You can use functions that simplify the search of optimal parameters, such as caret in (R ) or GridSearchCV in scikit learn


#### 2. Train ML model D~X and calculate residuals using cross-fitting
- Execute cross-validation (train-test split) over a subset of hyperparameters, and choose the one with lower error. You can use functions that simplify the search of optimal parameters, such as caret in (R ) or GridSearchCV in scikit learn


#### 3. Run linear regression Yresiduals~ Dresiduals

Finally, check that the results are similar than the ones DoubleML would give (RHC data ATE estimation with DoubleML (Python).ipynb)

|        | $coef$         | $std\ err$    | $t$           | $P>\|t\|$       | $2.5%$        | $97.5%$       |
|:------:|:------------:|:-----------:|:-----------:|:-----------:|:-----------:|:-----------:|
| swang1 | 0.039061     | 0.013351    | 2.925709    | 0.003437    | 0.012894    | 0.065229    |

----------------



In [1]:
import pandas as pd
import yaml
import numpy as np
from numpy.random import seed
import matplotlib.pyplot as plt
import seaborn as sns

import doubleml as dml

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error as rmse
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
import random
from tabulate import tabulate

#warnings
import warnings
warnings.filterwarnings('ignore')

seed(123)

In [2]:
confounders = [
    "age", "sex", "race", "edu", "income", "ninsclas", "cat1", "das2d3pc", "dnr1", "ca",
    "surv2md1", "aps1", "scoma1", "wtkilo1", "temp1", "meanbp1", "resp1", "hrt1", "pafi1",
    "paco21", "ph1", "wblc1", "hema1", "sod1", "pot1", "crea1", "bili1", "alb1", "resp",
    "card", "neuro", "gastr", "renal", "meta", "hema", "seps", "trauma", "ortho", "cardiohx",
    "chfhx", "dementhx", "psychhx", "chrpulhx", "renalhx", "liverhx", "gibledhx", "malighx",
    "immunhx", "transhx", "amihx"
]

In [3]:
rhc = pd.read_csv("rhc.csv")
with open("confounders.yml", "r") as f:
    confounders = yaml.safe_load(f)

rhc["swang1"] = (rhc["swang1"] == "RHC").astype(int)
rhc["death"] = (rhc["death"] == "Yes").astype(int)
rhc.groupby("swang1")["death"].mean()

rhc_numerical = pd.get_dummies(rhc[['swang1', 'death'] + confounders], dtype=float, drop_first=True)
display(rhc_numerical)


,swang1,death,age,edu,das2d3pc,surv2md1,aps1,scoma1,wtkilo1,temp1,...,resp_Yes,card_Yes,neuro_Yes,gastr_Yes,renal_Yes,meta_Yes,hema_Yes,seps_Yes,trauma_Yes,ortho_Yes
0,0,0,70.25098,12.000000,23.50000,0.640991,46,0,64.69995,38.69531,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,78.17896,12.000000,14.75195,0.755000,50,0,45.69998,38.89844,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1,0,46.09198,14.069916,18.13672,0.317000,82,0,0.00000,36.39844,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,75.33197,9.000000,22.92969,0.440979,48,0,54.59998,35.79688,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,67.90997,9.945259,21.05078,0.437000,72,41,78.39996,34.79688,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5730,0,0,75.56195,6.000000,14.75195,0.544000,45,9,78.19995,39.79688,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5731,0,1,44.65698,12.000000,23.00000,0.637000,62,0,50.50000,39.09375,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5732,0,1,80.48499,17.000000,17.50000,0.740000,43,0,86.00000,38.00000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5733,0,1,67.37897,12.000000,16.05859,0.671997,51,9,98.00000,38.79688,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
rhc.head(5)

,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,...,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,COPD,NaN,Yes,11142,11151.0,NaN,11382,0,0,...,No,No,No,No,No,0.0,NaN,white,Under $11k,5
1,2,MOSF w/Sepsis,NaN,No,11799,11844.0,11844.0,11844,1,1,...,No,No,Yes,No,No,NaN,1437.0,white,Under $11k,7
2,3,MOSF w/Malignancy,MOSF w/Sepsis,Yes,12083,12143.0,NaN,12400,0,0,...,No,No,No,No,No,NaN,599.0,white,$25-$50k,9
3,4,ARF,NaN,No,11146,11183.0,11183.0,11182,1,0,...,No,No,No,No,No,NaN,NaN,white,$11-$25k,10
4,5,MOSF w/Sepsis,NaN,No,12035,12037.0,12037.0,12036,1,0,...,No,No,No,No,No,NaN,64.0,white,Under $11k,11


In [5]:
rhc.shape

(5735, 63)

# Analysis
---------

### 1. Regress with DEATH as target


In [6]:
fold1, fold2= next(KFold(n_splits = 2, shuffle = True, random_state = 123).split(rhc_numerical))

print(fold1)
print(fold2)

[   0    1    2 ... 5730 5731 5732]
[   4    6    9 ... 5728 5733 5734]


In [7]:
df = rhc_numerical.drop('swang1', axis = 1).copy()
target = df['death'].copy()

X = df.drop('death', axis=1).copy()
y = target.copy()

In [8]:
# Step 1: Define the number of folds for cross-fitting
n_splits = 2

In [9]:
# Step 2: Define the model
rf = RandomForestClassifier()

In [10]:
# Step 3: Define hyperparameters to search
param_grid = {
    'n_estimators': [150, 200, 250],  # Number of trees in the forest
    'max_depth': [1, 2, 5],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 8],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 5],  # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt'],  # The number of features to consider when looking for the best split
    'bootstrap': [True, False],  # Whether bootstrap samples are used when building trees
    'criterion': ['gini', 'entropy']  # The function to measure the quality of a split
}

X_train1, X_test1 = X.iloc[fold1], X.iloc[fold2]
y_train1, y_test1 = y.iloc[fold1], y.iloc[fold2]

grid_search1 = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='roc_auc', n_jobs=-1, verbose=3)
grid_search1.fit(X_train1, y_train1)

# SAVING THE BEST MODEL
best_rf1 = grid_search1.best_estimator_

# TRAINING THE BEST MODEL
best_rf1.fit(X_train1, y_train1)

# FITTING THE BEST MODEL ON THE TEST SET
train_pred1 = best_rf1.predict_proba(X_train1)[:, 1]
test_pred1 = best_rf1.predict_proba(X_test1)[:, 1]

train_auc1 = roc_auc_score(y_train1, train_pred1)
test_auc1 = roc_auc_score(y_test1, test_pred1)


Fitting 3 folds for each of 648 candidates, totalling 1944 fits


[CV 1/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=nan total time=   0.0s
[CV 1/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=250;, score=nan total time=   0.0s
[CV 2/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=nan total time=   0.0s
[CV 3/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=250;, score=nan total time=   0.0s
[CV 3/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=nan total time=   0.0s
[CV 1/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=nan total time=   0.0s
[CV 

#### DEATH: Results 1

In [11]:
print("Train AUC:", train_auc1)
print("Test AUC:", test_auc1)
print("Best Parameters: ", best_rf1.get_params())

residuals_Y1 = y_test1 - test_pred1

print("Residuals for Y ~ X on fold 1 as train:", residuals_Y1)

Train AUC: 0.8240990967028057
Test AUC: 0.7618338280839754
Best Parameters:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 150, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
Residuals for Y ~ X on fold 1 as train: 4       0.191350
6      -0.802460
9      -0.632991
10     -0.306104
11     -0.489135
          ...   
5726    0.362078
5727    0.240839
5728    0.360235
5733    0.366287
5734    0.263242
Name: death, Length: 2868, dtype: float64


In [12]:
X_train2, X_test2 = X.iloc[fold2], X.iloc[fold1]
y_train2, y_test2 = y.iloc[fold2], y.iloc[fold1]

grid_search2 = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='roc_auc', n_jobs=-1, verbose=3)
grid_search2.fit(X_train2, y_train2)

# SAVING THE BEST MODEL
best_rf2 = grid_search2.best_estimator_

# TRAINING THE BEST MODEL
best_rf2.fit(X_train2, y_train2)

# FITTING THE BEST MODEL ON THE TEST SET
train_pred2 = best_rf2.predict_proba(X_train2)[:, 1]
test_pred2 = best_rf2.predict_proba(X_test2)[:, 1]

train_auc2 = roc_auc_score(y_train2, train_pred2)
test_auc2 = roc_auc_score(y_test2, test_pred2)



Fitting 3 folds for each of 648 candidates, totalling 1944 fits
[CV 1/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=nan total time=   0.0s
[CV 2/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=nan total time=   0.0s
[CV 3/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=nan total time=   0.0s
[CV 1/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=nan total time=   0.0s
[CV 2/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=nan total time=   0.0s
[CV 3/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_sampl

#### DEATH: Results 2

In [13]:
print("Train AUC:", train_auc2)
print("Test AUC:", test_auc2)
print("Best Parameters: ", best_rf2.get_params())

residuals_Y2 = y_test2 - test_pred2

print("Residuals for Y ~ X on fold 1 as train:", residuals_Y2)

Train AUC: 0.8284370880799031
Test AUC: 0.7546569279383758
Best Parameters:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 200, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
Residuals for Y ~ X on fold 1 as train: 0      -0.657208
1       0.384811
2      -0.906006
3       0.292930
5      -0.638017
          ...   
5725   -0.547647
5729   -0.702612
5730   -0.696819
5731    0.460184
5732    0.393716
Name: death, Length: 2867, dtype: float64


### 2. Regress with SWANG1 as target

In [14]:
df = rhc_numerical.drop('death', axis = 1).copy()
#display(df)
target = df['swang1'].copy()

X = df.drop('swang1', axis=1).copy()
y = target.copy()

In [15]:
# Step 1: Define the number of folds for cross-fitting
n_splits = 2
kf = KFold(n_splits=n_splits, shuffle=True, random_state=123)

In [16]:
# Step 2: Define the model
rf = RandomForestClassifier()

In [17]:
# Step 3: Define hyperparameters to search
param_grid = {
    'n_estimators': [150, 200, 250],  # Number of trees in the forest
    'max_depth': [1, 2, 5],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 8],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 5],  # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt'],  # The number of features to consider when looking for the best split
    'bootstrap': [True, False],  # Whether bootstrap samples are used when building trees
    'criterion': ['gini', 'entropy']  # The function to measure the quality of a split
}

X_train1, X_test1 = X.iloc[fold1], X.iloc[fold2]
y_train1, y_test1 = y.iloc[fold1], y.iloc[fold2]

grid_search1 = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='roc_auc', n_jobs=-1, verbose=3)
grid_search1.fit(X_train1, y_train1)

# SAVING THE BEST MODEL
best_rf1 = grid_search1.best_estimator_

# TRAINING THE BEST MODEL
best_rf1.fit(X_train1, y_train1)

# FITTING THE BEST MODEL ON THE TEST SET
train_pred1 = best_rf1.predict_proba(X_train1)[:, 1]
test_pred1 = best_rf1.predict_proba(X_test1)[:, 1]

train_auc1 = roc_auc_score(y_train1, train_pred1)
test_auc1 = roc_auc_score(y_test1, test_pred1)



Fitting 3 folds for each of 648 candidates, totalling 1944 fits
[CV 1/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=nan total time=   0.0s
[CV 2/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=nan total time=   0.0s
[CV 3/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=nan total time=   0.0s
[CV 1/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=nan total time=   0.0s
[CV 2/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=nan total time=   0.0s
[CV 3/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_sampl

#### SWANG1: Results 1

In [18]:
print("Train AUC:", train_auc1)
print("Test AUC:", test_auc1)
print("Best Parameters: ", best_rf1.get_params())

residuals_D1 = y_test1 - test_pred1

print("Residuals for Y ~ X on fold 1 as train:", residuals_D1)

Train AUC: 0.8534344545338279
Test AUC: 0.7652423650923911
Best Parameters:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 250, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
Residuals for Y ~ X on fold 1 as train: 4       0.443075
6      -0.330262
9       0.671237
10     -0.331088
11      0.592356
          ...   
5726   -0.345768
5727   -0.568087
5728    0.526949
5733   -0.386642
5734   -0.498238
Name: swang1, Length: 2868, dtype: float64


In [19]:
X_train2, X_test2 = X.iloc[fold2], X.iloc[fold1]
y_train2, y_test2 = y.iloc[fold2], y.iloc[fold1]

grid_search2 = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='roc_auc', n_jobs=-1, verbose=3)
grid_search2.fit(X_train2, y_train2)

# SAVING THE BEST MODEL
best_rf2 = grid_search2.best_estimator_

# TRAINING THE MODEL
best_rf2.fit(X_train2, y_train2)

# FITTING THE MODEL ON TRAINING DATA
train_pred2 = best_rf2.predict_proba(X_train2)[:, 1]
test_pred2 = best_rf2.predict_proba(X_test2)[:, 1]

train_auc2 = roc_auc_score(y_train2, train_pred2)
test_auc2 = roc_auc_score(y_test2, test_pred2)



Fitting 3 folds for each of 648 candidates, totalling 1944 fits
[CV 1/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=nan total time=   0.0s
[CV 2/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=nan total time=   0.0s
[CV 3/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=nan total time=   0.0s
[CV 1/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=nan total time=   0.0s
[CV 2/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=nan total time=   0.0s
[CV 3/3] END bootstrap=True, criterion=gini, max_depth=1, max_features=auto, min_samples_leaf=1, min_sampl

#### SWANG1: Results 2

In [20]:
print("Train AUC:", train_auc2)
print("Test AUC:", test_auc2)
print("Best Parameters: ", best_rf2.get_params())

residuals_D2 = y_test2 - test_pred2

print("Residuals for Y ~ X on fold 1 as train:", residuals_D2)

Train AUC: 0.861855357874179
Test AUC: 0.7875579997442129
Best Parameters:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 8, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 250, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
Residuals for Y ~ X on fold 1 as train: 0      -0.408261
1       0.581106
2       0.560023
3      -0.343470
5      -0.205730
          ...   
5725    0.520570
5729   -0.378171
5730   -0.447330
5731   -0.426103
5732   -0.192309
Name: swang1, Length: 2867, dtype: float64


In [21]:
res_D1 = residuals_D1.copy()
res_Y1 = residuals_Y1.copy()

# Reshape residuals_D for statsmodels compatibility (as a feature matrix)
X_residuals_D1 = sm.add_constant(res_D1.values.reshape(-1, 1))  # Adding a constant for intercept
y_residuals_Y1 = res_Y1.values

# Linear Regression using statsmodels
linear_model_sm1 = sm.OLS(y_residuals_Y1, X_residuals_D1)
results_linear1 = linear_model_sm1.fit()
print(results_linear1.summary())

# Store coefficients and standard errors
coefficients1 = results_linear1.params[1]
std_errors1 = results_linear1.bse[1]

# You can also store other statistics as needed
t_values1 = results_linear1.tvalues[1]
p_values1 = results_linear1.pvalues[1]
conf_int1 = results_linear1.conf_int()[1, :]

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.064
Date:                Sun, 25 Feb 2024   Prob (F-statistic):             0.0245
Time:                        18:26:38   Log-Likelihood:                -1679.2
No. Observations:                2868   AIC:                             3362.
Df Residuals:                    2866   BIC:                             3374.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0040      0.008      0.490      0.6

In [22]:
res_D2 = residuals_D2.copy()
res_Y2 = residuals_Y2.copy()

# Reshape residuals_D for statsmodels compatibility (as a feature matrix)
X_residuals_D2 = sm.add_constant(res_D2.values.reshape(-1, 1))  # Adding a constant for intercept
y_residuals_Y2 = res_Y2.values

# Linear Regression using statsmodels
linear_model_sm2 = sm.OLS(y_residuals_Y2, X_residuals_D2)
results_linear2 = linear_model_sm2.fit()
print(results_linear2.summary())

# Store coefficients and standard errors
coefficients2 = results_linear2.params[1]
std_errors2 = results_linear2.bse[1]

# You can also store other statistics as needed
t_values2 = results_linear2.tvalues[1]
p_values2 = results_linear2.pvalues[1]
conf_int2 = results_linear2.conf_int()[1, :]

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.663
Date:                Sun, 25 Feb 2024   Prob (F-statistic):              0.103
Time:                        18:26:38   Log-Likelihood:                -1702.1
No. Observations:                2867   AIC:                             3408.
Df Residuals:                    2865   BIC:                             3420.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0010      0.008     -0.127      0.8

# Results

In [23]:
avg_coeff = np.mean([coefficients1, coefficients2])
avg_std = np.mean([std_errors1, std_errors2])
avg_t = np.mean([t_values1, t_values2])
avg_p = np.mean([p_values1, p_values2])
avg_confmin = np.mean([conf_int1[0], conf_int2[0]])
avg_confmax = np.mean([conf_int1[1], conf_int2[1]])

print("coef:", avg_coeff)
print("std err:", avg_std)
print("t:", avg_t)
print("P > t:", avg_p)
print("2.5", avg_confmin)
print("97.5", avg_confmax)

coef: 0.03568439048928361
std err: 0.018392929749170065
t: 1.9411168229179205
P > t: 0.06366433796034486
2.5 -0.00038032274007517193
97.5 0.0717491037186424


In [26]:
# Table of results
#create data
data_table = [[avg_coeff, avg_std, avg_t , avg_p , avg_confmin , avg_confmax]]

#define header names
col_names = ["coef", "std err", "t", "P > t", "2.5", "9.75"]

#display table
print('Results of the average of the two folds:')
print('__________________________________________________________________')
print(tabulate(data_table, headers=col_names))
print('__________________________________________________________________')

Results of the average of the two folds:
__________________________________________________________________
     coef    std err        t      P > t           2.5       9.75
---------  ---------  -------  ---------  ------------  ---------
0.0356844  0.0183929  1.94112  0.0636643  -0.000380323  0.0717491
__________________________________________________________________


Finally, check that the results are similar than the ones DoubleML would give (RHC data ATE estimation with DoubleML (Python).ipynb)

|        | $coef$         | $std\ err$    | $t$           | $P>\|t\|$       | $2.5%$        | $97.5%$       |
|:------:|:------------:|:-----------:|:-----------:|:-----------:|:-----------:|:-----------:|
| swang1 | 0.039061     | 0.013351    | 2.925709    | 0.003437    | 0.012894    | 0.065229    |

----------------